In [1]:
import os, sys
import numpy as np
import torch
import torchaudio
import matplotlib.pyplot as plt
import IPython.display as ipd
from asteroid.metrics import get_metrics
from pprint import pprint
import asteroid
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
torch.cuda.empty_cache()
sys.path.append("sudo_rm_rf/")

# Get the pretrained models
print("Pre-trained models available:")
for model_name in os.listdir('sudo_rm_rf/pretrained_models'):
    print(model_name)
    
def normalize_tensor_wav(wav_tensor, eps=1e-8, std=None):
    mean = wav_tensor.mean(-1, keepdim=True)
    if std is None:
        std = wav_tensor.std(-1, keepdim=True)
    return (wav_tensor - mean) / (std + eps)
    
anechoic_model_p = 'sudo_rm_rf/pretrained_models/GroupCom_Sudormrf_U8_Bases512_WSJ02mix.pt'
anechoic_model_p = 'sudo_rm_rf/pretrained_models/Improved_Sudormrf_U16_Bases512_WSJ02mix.pt'
anechoic_model_p = 'sudo_rm_rf/pretrained_models/Improved_Sudormrf_U36_Bases2048_WSJ02mix.pt'
noisy_reverberant_model_p = 'sudo_rm_rf/pretrained_models/Improved_Sudormrf_U16_Bases2048_WHAMRexclmark.pt'
noisy_reverberant_model_p = 'sudo_rm_rf/pretrained_models/Improved_Sudormrf_U36_Bases4096_WHAMRexclmark.pt'

# Load the appropriate class modules
whamr_test_folder_path = 'data/8000'
import sudo_rm_rf.dnn.experiments.utils.mixture_consistency as mixture_consistency
import sudo_rm_rf.dnn.models.improved_sudormrf as improved_sudormrf
import sudo_rm_rf.dnn.models.groupcomm_sudormrf_v2 as sudormrf_gc_v2

Pre-trained models available:
.gitattributes
download_pretrained_models.sh


In [2]:
# Just to understand the difference, play the output of a few different models
all_anechoic_models_paths = [
    'sudo_rm_rf/pretrained_models/GroupCom_Sudormrf_U8_Bases512_WSJ02mix.pt',
    'sudo_rm_rf/pretrained_models/Improved_Sudormrf_U16_Bases512_WSJ02mix.pt',
    'sudo_rm_rf/pretrained_models/Improved_Sudormrf_U36_Bases2048_WSJ02mix.pt',
    'sudo_rm_rf/pretrained_models/Improved_Sudormrf_U36_Bases4096_WHAMRexclmark.pt']



FILES_PATH_INPUT = 'data_pipeline/8000_1/DPTNET/'
FILES_INPUT = os.listdir(FILES_PATH_INPUT)
for chosen_file in FILES_INPUT:
    wav_path = chosen_file

    anechoic_sampled_mixture, _ = torchaudio.load(FILES_PATH_INPUT+wav_path)

    waveform = anechoic_sampled_mixture.detach().numpy()[0]


    for anechoic_model_p in all_anechoic_models_paths:
        name_folder = "_".join(anechoic_model_p.split('/')[-1].split('_')[2:4])
        if not os.path.isdir('data_pipeline/8000_1/'+"DPTNET_"+name_folder):
            os.mkdir('data_pipeline/8000_1/'+"DPTNET_"+name_folder)
        model_name = os.path.basename(anechoic_model_p)
        print(model_name)

        # Load a pretrained model
        anechoic_separation_model = torch.load(anechoic_model_p)

        # Normalize the waveform and apply the model
        input_mix_std = anechoic_sampled_mixture.std(-1, keepdim=True)
        input_mix_mean = anechoic_sampled_mixture.mean(-1, keepdim=True)
        input_mix = (anechoic_sampled_mixture - input_mix_mean) / (input_mix_std + 1e-9)

        # Apply the model
        with torch.no_grad():
            rec_sources_wavs = anechoic_separation_model(input_mix.unsqueeze(1))

            # # In case you are using the pre-trained models with Group communication
            # # please also use the mixture consistency right after the estimated waveforms
            if "GroupCom" in model_name:
                rec_sources_wavs = mixture_consistency.apply(rec_sources_wavs, input_mix.unsqueeze(1))

            # Rescale the input sources with the mixture mean and variance
            rec_sources_wavs = (rec_sources_wavs * input_mix_std) + input_mix_mean

        torchaudio.save('data_pipeline/8000_1/'+'DPTNET_'+name_folder+"/"+chosen_file, normalize_tensor_wav(rec_sources_wavs[0]).detach(),8000)

GroupCom_Sudormrf_U8_Bases512_WSJ02mix.pt


FileNotFoundError: [Errno 2] No such file or directory: 'sudo_rm_rf/pretrained_models/GroupCom_Sudormrf_U8_Bases512_WSJ02mix.pt'